In [1]:
from src.model_train import ModelTrainer, validate_folds
import pandas as pd

from cashe import get_training_inputs, drop_nomodel_columns, get_train_test_split
from src.data_local import MergedDataPreprocessing,read_last_date, append_last_line
from src.lstm_encoder import LSTMEmbedding
from src.model_app import ModelApplication

In [2]:
df_train, df_test = get_train_test_split(path='data/HJH/13-06-2024')

In [3]:
df_train.head()

,REQUEST_DATE,VISIT_NO,VISIT_DATE,EMERGENCY_INDICATOR,PROVIDER_DEPARTMENT_CODE,PROVIDER_DEPARTMENT,DOCTOR_SPECIALTY_CODE,DOCTOR_CODE,PATIENT_NO,EPISODE_NO,...,SEQUENCE_NO,UNIT_PRICE_STOCKED_UOM,UNIT_PRICE_NET,DISCOUNT_PERCENTAGE,NOTES,OUTCOME,APPROVED_QUNATITY,Episode_Key,ICD10,Diagnosis_Key
398448,2024-01-01 12:33:25,1346536-2,2023-12-07,N,855,CARDIOTHORACIC,8.25,8455,1183018,2,...,1,120.00,96.00,0.00,,APPROVED,1.0,1_1346536-2,J93,1_1111225808
398449,2024-01-01 12:38:12,1343530-4,2023-12-06,N,4,DENTAL SERVICES - اسنان,22.00,7352,1180321,4,...,1,120.00,120.00,0.00,Rejected for the reason Sorry you are not an a...,REJECTED,0.0,1_1343530-4,K02,1_1111225570
398453,2024-01-01 12:38:36,1343731-2,2023-12-03,N,25,RADIOGRAPHIC INVESTIGATIONS - الأشعة,18.06,9638,1180507,2,...,4,1791.67,1154.17,30.00,Rejected for the reason Sorry you are not an a...,REJECTED,0.0,1_1343731-2,N80.1,1_1111228814
398454,2024-01-01 12:38:36,1343731-2,2023-12-03,N,25,RADIOGRAPHIC INVESTIGATIONS - الأشعة,18.06,9638,1180507,2,...,5,1.69,1.25,7.69,Rejected for the reason Sorry you are not an a...,REJECTED,0.0,1_1343731-2,N80.1,1_1111228814
398450,2024-01-01 12:38:36,1343731-2,2023-12-03,N,25,RADIOGRAPHIC INVESTIGATIONS - الأشعة,18.06,9638,1180507,2,...,1,16.13,11.29,30.01,Rejected for the reason Sorry you are not an a...,REJECTED,0.0,1_1343731-2,N80.1,1_1111228814


In [4]:
df_test.CREATION_DATE.sort_values() ## ranges in the last 2 months only after updating split func.

316188   2024-04-06 13:14:51
316187   2024-04-06 13:14:51
316186   2024-04-06 13:14:51
301026   2024-04-06 13:15:27
301402   2024-04-06 13:15:27
                 ...        
647258   2024-05-30 11:46:53
648079   2024-05-30 11:46:53
648044   2024-05-30 11:46:53
647256   2024-05-30 11:46:53
648062   2024-05-30 11:46:53
Name: CREATION_DATE, Length: 41396, dtype: datetime64[ns]

In [22]:
#read_last_date()

In [5]:
X_train, y_train, X_test, y_test = get_training_inputs(df_train, df_test)

In [6]:
preprocessing_train = MergedDataPreprocessing(X_train)
X_train_prep = preprocessing_train.columns_prep()

preprocessing_test = MergedDataPreprocessing(X_test)
X_test_prep = preprocessing_test.columns_prep()

E:\Projects\Claims_Rejection\src\data_local.py:276: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column]= df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:284: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PatientAgeRange']= df['PatientAgeRange'].replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:276: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(cop

In [7]:
lstm_embedding = LSTMEmbedding()

X_train_encoded = preprocessing_train.column_embedding(X_train_prep, is_service=True)
X_test_encoded = preprocessing_train.column_embedding(X_test_prep, is_service=True)

X_train_encoded = preprocessing_train.column_embedding(X_train_encoded, is_service=False)
X_test_encoded = preprocessing_train.column_embedding(X_test_encoded, is_service=False)

X_train = drop_nomodel_columns(X_train_encoded)
X_test  = drop_nomodel_columns(X_test_encoded)

In [8]:
X_train

,EMERGENCY_INDICATOR,PROVIDER_DEPARTMENT_CODE,DOCTOR_SPECIALTY_CODE,DOCTOR_CODE,PATIENT_AGE,UNIT_OF_AGE,PATIENT_NATIONALITY,PATIENT_MARITAL_STATUS,PATIENT_GENDER,CLAIM_TYPE,...,ICDText7,ICDText8,ICDText9,ICDText10,ICDText11,ICDText12,ICDText13,ICDText14,ICDText15,ICDText16
398448,0,855,8.25,8455,43,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
398449,0,4,22.00,7352,0,2,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
398453,0,25,18.06,9638,33,1,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
398454,0,25,18.06,9638,33,1,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
398450,0,25,18.06,9638,33,1,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301022,0,29,8.02,3396,45,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
301021,0,29,8.02,3396,45,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
301023,0,29,8.02,3396,45,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
301020,0,29,8.02,3396,45,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
model_runner = ModelApplication(X_train= X_train,y_train= y_train['OUTCOME'],X_test= X_test,y_test= y_test['OUTCOME'])

In [10]:
model_runner.get_model(force_retrain=True,model_type='outcome')

Training is done


In [11]:
## runtime 1
model_runner.evaluate_model()

{'XGBoost': {'Accuracy': 0.76,
  'Precision': 0.71,
  'Recall': 0.79,
  'F1 Score': 0.75}}

In [12]:
## runtime 2
model_runner.evaluate_model()

{'XGBoost': {'Accuracy': 0.76,
  'Precision': 0.71,
  'Recall': 0.79,
  'F1 Score': 0.75}}

## Columns removal comparison

In [13]:
## before the high correlation appliance
model_runner = ModelApplication(X_train=X_train, y_train=y_train['OUTCOME'], X_test=X_test, y_test=y_test['OUTCOME'])
model_runner.get_model(force_retrain=False, model_type='outcome')
model_runner.evaluate_model()

XGBoost Model is loading from disk..


{'XGBoost': {'Accuracy': 0.76,
  'Precision': 0.71,
  'Recall': 0.79,
  'F1 Score': 0.75}}

In [14]:
def find_service_importances(model_features):
    combined_text_rows = model_features[model_features['Column name'].str.contains('CombinedText')]
    combined_sum = sum(combined_text_rows['Importance'])
    model_features = model_features[~model_features['Column name'].str.contains('CombinedText')]
    model_features.loc[len(model_features)] = {'Column name':'SERVICE_DESCRIPTION','Importance':float(combined_sum)}
    return model_features

def find_icd_importances(model_features):
    combined_text_rows = model_features[model_features['Column name'].str.contains('ICDText')]
    combined_sum = sum(combined_text_rows['Importance'])
    model_features = model_features[~model_features['Column name'].str.contains('ICDText')]
    model_features.loc[len(model_features)] = {'Column name':'ICD_10','Importance':float(combined_sum)}
    model_features.sort_values(by=['Importance'],ascending=False,inplace=True)
    return model_features

In [15]:
features_imp = list(model_runner.get_feature_importance())
df_results = pd.DataFrame({"Column name":list(X_train.columns),"Importance":features_imp}).sort_values(ascending=False,by=['Importance'])
df_results

,Column name,Importance
27,DURATION,0.075301
14,DEPARTMENT_TYPE,0.071871
25,VAT_PERCENTAGE,0.051161
16,PURCHASER_CODE,0.049959
56,ICDText7,0.049661
...,...,...
33,PatientAgeRange,0.000000
60,ICDText11,0.000000
57,ICDText8,0.000000
26,TREATMENT_TYPE_INDICATOR,0.000000


In [16]:
df_results = find_service_importances(df_results)
df_results = find_icd_importances(df_results)
df_results.to_excel('FeatureImportance_claimrej.xlsx',index=False)

C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_12604\940300331.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1863840336445719' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  model_features.loc[len(model_features)] = {'Column name':'SERVICE_DESCRIPTION','Importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_12604\940300331.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column name':'ICD_10','Importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_12604\940300331.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

In [21]:
cols_drop = ['CLAIM_INVOICE_NO','STAT_INVOICE_NO','CLAIM_INVOICE_NO','HIS_INSURANCE_CODE','CREATED_BY_USER','TOTAL_NET_AMOUNT',
             'TOTAL_NET_VAT_AMOUNT','TOTAL_CLAIMED_AMOUNT','LINE_CLAIMED_AMOUNT','CO_INSURANCE','NET_AMOUNT','UNIT_PRICE',
             'CO_PAY']

cols_kept = [col for col in X_train.columns if col not in cols_drop]

## after the high correlation appliance
model_runner = ModelApplication(X_train=X_train[cols_kept], y_train=y_train['OUTCOME'],
                                X_test=X_test[cols_kept], y_test=y_test['OUTCOME'])
model_runner.get_model(force_retrain=True, model_type='outcome')
model_runner.evaluate_model()

Training is done


{'XGBoost': {'Accuracy': 0.76,
  'Precision': 0.71,
  'Recall': 0.78,
  'F1 Score': 0.75}}

In [23]:
features_imp = list(model_runner.get_feature_importance())
pd.DataFrame({"Column name":list(cols_kept),"Importance":features_imp}).sort_values(ascending=False,by=['Importance']).to_excel('FeatureImportance_afterdrop.xlsx',index=False)

## Features F1 Scores comparison

In [55]:
import pandas as pd
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

def feature_importance_by_f1(X_train, X_test, y_train, y_test):
    y_train, y_test = list(y_train), list(y_test)
    model = XGBClassifier()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    baseline_f1 = f1_score(y_test, y_pred, average='weighted')

    feature_f1_diffs = {}

    # Iterate over each feature
    for feature in X_train.columns:
        X_train_dropped = X_train.drop(columns=[feature]) ## drop only one feature per time
        X_test_dropped = X_test.drop(columns=[feature])

        model.fit(X_train_dropped, y_train)
        y_pred_dropped = model.predict(X_test_dropped)
        f1_dropped = f1_score(y_test, y_pred_dropped, average='weighted')
        # Calculate the difference in F1 score
        f1_diff = baseline_f1 - f1_dropped
        if 'ICDText' not in feature and 'CombinedText' not in feature: ## these features don't have a singular meaning
            feature_f1_diffs[feature] = f1_diff
            if f1_diff < 0:
                    print(f"Feature: {feature}, F1 score difference: {f1_diff}")

    sorted_features = sorted(feature_f1_diffs.items(), key=lambda item: item[1], reverse=True)
    return pd.DataFrame(sorted_features, columns=['Feature', 'F1 Difference'])

In [56]:
df_features_elimination = feature_importance_by_f1(X_train, X_test, y_train['OUTCOME'], y_test.OUTCOME)

Feature: CO_INSURANCE, F1 score difference: -0.00036586662936255454
Feature: EMERGENCY_INDICATOR, F1 score difference: -0.0004249761190200374
Feature: PATIENT_GENDER, F1 score difference: -0.002823602412574955
Feature: TOTAL_DEDUCTIBLE, F1 score difference: -0.001787388894478159


In [57]:
df_features_elimination

,Feature,F1 Difference
0,PURCHASER_CODE,0.021611
1,DISCOUNT_PERCENTAGE,0.010042
2,DOCTOR_SPECIALTY_CODE,0.010007
3,CONTRACT_NO,0.009014
4,PATIENT_MARITAL_STATUS,0.006719
5,UNIT_PRICE_NET,0.005967
6,PROVIDER_DEPARTMENT_CODE,0.005610
7,CLAIM_TYPE,0.004818
8,TREATMENT_TYPE,0.004171
9,DURATION,0.003587


## Other old trials

In [14]:
## Comparison between the embedding vector added columns
print('The result before embedding the Item_Name column are:\n',res_before,'\nThe results after the embedding:\n',res)

The result before embedding the Item_Name column are:
 {'Decision Tree': {'Accuracy': 0.67, 'Precision': 0.66, 'Recall': 0.64}, 'LightGBM': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.71}, 'SGD Classifier': {'Accuracy': 0.52, 'Precision': 0.0, 'Recall': 0.0}, 'XGBoost': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.73}, 'Neural Network': {'Accuracy': 0.56, 'Precision': 0.55, 'Recall': 0.44}} 
The results after the embedding:
 {'Decision Tree': {'Accuracy': 0.67, 'Precision': 0.66, 'Recall': 0.63}, 'LightGBM': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.71}, 'SGD Classifier': {'Accuracy': 0.48, 'Precision': 0.48, 'Recall': 0.9}, 'XGBoost': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.73}, 'Neural Network': {'Accuracy': 0.55, 'Precision': 0.57, 'Recall': 0.28}}


In [16]:
## Feature importance Case Study
dt_tran = model_runner.get_decision_tree_feature_importance()
dt_feats = pd.DataFrame(list(dt_tran),columns_needed)

dt_feats[0].sort_values(ascending=False)

item_Price                     0.323421
transaction_PatientAge         0.313583
item_Sequence                  0.191764
transaction_DiagnosisIds       0.114149
transaction_PatientEnGender    0.044771
item_RequestQuantity           0.012312
Name: 0, dtype: float64

In [17]:
gbm_tran = model_runner.get_lightgbm_feature_importance()
gbm_feats = pd.DataFrame(list(gbm_tran),columns_needed)

gbm_feats[0].sort_values(ascending=False)

item_Price                     0.410000
transaction_PatientAge         0.255333
transaction_DiagnosisIds       0.137667
item_Sequence                  0.135333
item_RequestQuantity           0.031000
transaction_PatientEnGender    0.030667
Name: 0, dtype: float64

In [18]:
nn_tran = model_runner.get_neural_network_feature_importance()
nn_feats = pd.DataFrame(list(nn_tran),columns_needed)

nn_feats[0].sort_values(ascending=False)

item_Price                     7.984434e-02
item_RequestQuantity           5.211179e-02
item_Sequence                  3.358094e-02
transaction_DiagnosisIds       2.544493e-02
transaction_PatientAge         2.421929e-02
transaction_PatientEnGender    1.265203e-53
Name: 0, dtype: float64

## K-Fold Analysis

In [2]:
df_original = get_input()
preprocessing = MergedDataPreprocessing(df_original)

In [11]:
df_acc = validate_folds(df_original)



LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7437, number of negative: 7506
[LightGBM] [Info] Total Bins 1303
[LightGBM] [Info] Number of data points in the train set: 14943, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497691 -> initscore=-0.009235
[LightGBM] [Info] Start training from score -0.009235


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7480, number of negative: 7714
[LightGBM] [Info] Total Bins 1317
[LightGBM] [Info] Number of data points in the train set: 15194, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.492300 -> initscore=-0.030804
[LightGBM] [Info] Start training from score -0.030804


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7558, number of negative: 7626
[LightGBM] [Info] Total Bins 1298
[LightGBM] [Info] Number of data points in the train set: 15184, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497761 -> initscore=-0.008957
[LightGBM] [Info] Start training from score -0.008957


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7480, number of negative: 7518
[LightGBM] [Info] Total Bins 1300
[LightGBM] [Info] Number of data points in the train set: 14998, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498733 -> initscore=-0.005067
[LightGBM] [Info] Start training from score -0.005067


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


In [12]:
df_acc

,Decision Tree,LightGBM,SGD Classifier,XGBoost,Neural Network
0,67,72,49,72,67
1,67,72,49,72,64
2,67,70,49,72,68
3,69,72,51,74,67
4,68,72,48,72,67


In [ ]:
## XGBoost 72.5% accuracy, Neural Networks 66%  accuracy which can definitely be improved